In [1]:
! pip install -q mlflow
import pandas as pd
from sklearn.impute import SimpleImputer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
! pip install -q mlflow
import pandas as pd
import os
import time
import pandas as pd
import seaborn as sns 
import mlflow
from mlflow.tracking import MlflowClient
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import mlflow
from sklearn.model_selection import GridSearchCV


%matplotlib inline


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
df= pd.read_csv("EGX-DATA.csv")

In [25]:
df.head(20)

,date,close,open,high,low,volume
0,2020-11-02T00:00:00-05:00,10429.3,10456.3,10511.5,10426.8,181857000.0
1,2020-11-03T00:00:00-05:00,10512.2,10429.3,10512.2,10426.9,114152000.0
2,2020-11-04T00:00:00-05:00,10556.9,10512.2,10562.8,10512.2,75910200.0
3,2020-11-05T00:00:00-05:00,10629.3,10556.9,10638.1,10513.6,89506500.0
4,2020-11-09T00:00:00-05:00,10838.7,10680.3,10838.7,10680.3,188228000.0
5,2020-11-10T00:00:00-05:00,11077.2,10838.7,11123.0,10838.7,206459000.0
6,2020-11-11T00:00:00-05:00,10997.6,11077.2,11077.2,10982.2,174249000.0
7,2020-11-12T00:00:00-05:00,11017.3,10997.6,11028.4,10997.6,213677000.0
8,2020-11-16T00:00:00-05:00,10991.6,10953.1,10994.4,10953.1,145170000.0
9,2020-11-17T00:00:00-05:00,11064.0,10991.6,11076.7,10991.6,234763000.0


In [26]:
df.describe()

,close,open,high,low,volume
count,1085.000000,1085.000000,1085.000000,1085.000000,1.085000e+03
mean,21038.332576,20980.504105,21197.729677,20876.134192,1.638954e+08
std,9477.114204,9474.958012,9547.303538,9408.222762,8.359985e+07
min,8657.500000,0.000000,8708.970000,8598.430000,3.679440e+07
25%,11235.600000,11190.800000,11291.400000,11143.600000,1.031880e+08
50%,17921.900000,17921.900000,18071.500000,17872.600000,1.475320e+08
75%,30497.099610,30443.400000,30696.000000,30227.100000,2.066180e+08
max,38304.900000,38232.700000,38409.300000,38130.000000,7.578360e+08


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1085 entries, 0 to 1084
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    1085 non-null   object 
 1   close   1085 non-null   float64
 2   open    1085 non-null   float64
 3   high    1085 non-null   float64
 4   low     1085 non-null   float64
 5   volume  1085 non-null   float64
dtypes: float64(5), object(1)
memory usage: 51.0+ KB


In [36]:
# df['date'] = df['date'].str.replace('T00:00:00-05:00', '', regex=False)
# df['date'] = df['date'].str.replace('T00:00:00-04:00', '', regex=False)
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df["year"]= df['date'].dt.year
df["month"]= df['date'].dt.month
df["day"]= df['date'].dt.day
df.drop('date', axis=1 , inplace=True )


df.head(60)

,close,open,high,low,volume,year,month,day
0,10429.3,10456.3,10511.5,10426.8,181857000.0,2020,11,2
1,10512.2,10429.3,10512.2,10426.9,114152000.0,2020,11,3
2,10556.9,10512.2,10562.8,10512.2,75910200.0,2020,11,4
3,10629.3,10556.9,10638.1,10513.6,89506500.0,2020,11,5
4,10838.7,10680.3,10838.7,10680.3,188228000.0,2020,11,9
5,11077.2,10838.7,11123.0,10838.7,206459000.0,2020,11,10
6,10997.6,11077.2,11077.2,10982.2,174249000.0,2020,11,11
7,11017.3,10997.6,11028.4,10997.6,213677000.0,2020,11,12
8,10991.6,10953.1,10994.4,10953.1,145170000.0,2020,11,16
9,11064.0,10991.6,11076.7,10991.6,234763000.0,2020,11,17


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1085 entries, 0 to 1084
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   close   1085 non-null   float64
 1   open    1085 non-null   float64
 2   high    1085 non-null   float64
 3   low     1085 non-null   float64
 4   volume  1085 non-null   float64
 5   year    1085 non-null   int32  
 6   month   1085 non-null   int32  
 7   day     1085 non-null   int32  
dtypes: float64(5), int32(3)
memory usage: 55.2 KB


In [31]:
df.replace({',': ''}, regex=True , inplace=True)
df['volume ']=df['volume '].replace('M',' ', regex=True ).astype(float)
df['change'].replace('%','',inplace=True)


C:\Users\youss\AppData\Local\Temp\ipykernel_26752\2364614796.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['change'].replace('%','',inplace=True)


In [38]:
df.head()

,close,open,high,low,volume,year,month,day
0,10429.3,10456.3,10511.5,10426.8,181857000.0,2020,11,2
1,10512.2,10429.3,10512.2,10426.9,114152000.0,2020,11,3
2,10556.9,10512.2,10562.8,10512.2,75910200.0,2020,11,4
3,10629.3,10556.9,10638.1,10513.6,89506500.0,2020,11,5
4,10838.7,10680.3,10838.7,10680.3,188228000.0,2020,11,9


In [40]:
x= df.drop("close",axis =1)
y=df.close

In [41]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,train_size=0.8,random_state=35)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

param_grid = {
    "n_estimators": [100, 200, 300],
    "max_depth": [None, 3, 5, 7],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 2]
}

base_model = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(
    estimator=base_model,
    param_grid=param_grid,
    scoring="r2",  # Using R² score for regression
    cv=6,
    n_jobs=-1,
    verbose=1
)

# Fit Grid Search
grid_search.fit(xtrain, ytrain)

# Get the best model
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Evaluate best model
preds = best_model.predict(xtest)
mse = mean_squared_error(ytest, preds)
rmse = np.sqrt(mse)
r2 = r2_score(ytest, preds)

# Create scatter plot of predicted vs actual values
plt.figure(figsize=(10, 6))
plt.scatter(ytest, preds, alpha=0.5)
plt.plot([ytest.min(), ytest.max()], [ytest.min(), ytest.max()], 'r--', lw=2)
plt.xlabel('Actual Close Price')
plt.ylabel('Predicted Close Price')
plt.title('Random Forest Regression: Predicted vs Actual')
plt.savefig("prediction_scatter.png")
plt.close()

print("Best Parameters:", best_params)
print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R² Score: {r2:.2f}")
print(f"Model registered as version: ")

Fitting 2 folds for each of 48 candidates, totalling 96 fits
Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Mean Squared Error: 30513.46
Root Mean Squared Error: 174.68
R² Score: 1.00
Model registered as version: 
Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Mean Squared Error: 30513.46
Root Mean Squared Error: 174.68
R² Score: 1.00
Model registered as version: 


In [ ]:
# 1. Set MLflow Tracking URI
mlflow.set_tracking_uri("http://127.0.0.1:5000")

# 2. Set or create experiment
mlflow.set_experiment("stocks_regression")

# 3. Initialize MLflow client
client = MlflowClient()

# 4. Define model registry name and alias
model_name = "random_forest_regressor"
alias_name = "dev"  # change to "staging" or "prod" as needed

# 5. Define the parameter grid for Grid Search
param_grid = {
    "n_estimators": [100, 200, 300],
    "max_depth": [None, 3, 5, 7],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 2]
}

# 6. Start MLflow run
with mlflow.start_run(run_name=f"GridSearch for @{alias_name}"):
    
    # Set up Grid Search
    base_model = RandomForestRegressor(random_state=42)
    grid_search = GridSearchCV(
        estimator=base_model,
        param_grid=param_grid,
        scoring="r2",
        cv=2,
        n_jobs=-1,
        verbose=1
    )
    
    # Fit Grid Search
    grid_search.fit(xtrain, ytrain)

    # Get the best model
    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_

    # Evaluate best model
    preds = best_model.predict(xtest)
    mse = mean_squared_error(ytest, preds)
    rmse = np.sqrt(mse)
    r2 = r2_score(ytest, preds)

    # Log all hyperparameters and metrics
    mlflow.log_params(best_params)
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)

    # 7. Log Model
    mlflow.sklearn.log_model(best_model, artifact_path="model")

    # 8. Register model to MLflow Model Registry
    model_uri = f"runs:/{mlflow.active_run().info.run_id}/model"
    result = mlflow.register_model(model_uri=model_uri, name=model_name)
    version = result.version

    # Wait until model is ready
    for _ in range(10):
        model_info = client.get_model_version(model_name, version)
        if model_info.status == "READY":
            break
        time.sleep(2)

    # Assign alias (e.g., @dev)
    client.set_registered_model_alias(
        name=model_name,
        alias=alias_name,
        version=version
    )

    # Log dataset used
    df = pd.DataFrame(xtrain, columns=[f"feature_{i}" for i in range(xtrain.shape[1])])
    df["target"] = ytrain
    dataset_path = "egx_training_data.csv"
    df.to_csv(dataset_path, index=False)
    mlflow.log_artifact(dataset_path, artifact_path="dataset")

    # Log notebook or script if available
    if os.path.exists("Stocks.ipynb"):
        mlflow.log_artifact("Stocks.ipynb", artifact_path="code")

    # Create and log scatter plot
    plt.figure(figsize=(10, 6))
    plt.scatter(ytest, preds, alpha=0.5)
    plt.plot([ytest.min(), ytest.max()], [ytest.min(), ytest.max()], 'r--', lw=2)
    plt.xlabel('Actual Close Price')
    plt.ylabel('Predicted Close Price')
    plt.title('Random Forest Regression: Predicted vs Actual')
    plt.savefig("prediction_scatter.png")
    plt.close()
    mlflow.log_artifact("prediction_scatter.png", artifact_path="metrics")

    # Print summary
    print("Best Parameters:", best_params)
    print(f"Mean Squared Error: {mse:.2f}")
    print(f"Root Mean Squared Error: {rmse:.2f}")
    print(f"R² Score: {r2:.2f}")
    print(f"Model registered as version: {version}")
    print(f"Alias @{alias_name} now points to version: {version}")

Fitting 2 folds for each of 48 candidates, totalling 96 fits


c:\Users\youss\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(
2025/10/28 00:47:44 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/28 00:47:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
